In [ ]:
from IPython.core.display import HTML
with open('style.css', 'r') as file:
    css = file.read()
HTML(css)

# The 8-Queens Problem

The <a href="https://en.wikipedia.org/wiki/Eight_queens_puzzle">eight queens puzzle</a> is the problem of placing eight chess queens on a chessboard so that no two queens can capture each other.  In <a href="https://en.wikipedia.org/wiki/Chess">chess</a> a queen can capture another piece if this piece is either
<ol>
    <li>in the same row,</li>
    <li>in the same column, or</li>
    <li>in the same diagonal.</li>
</ol>
The image below shows a queen in row 3, column 4.  All the locations where a piece can be captured by this queen are marked with an arrow.

<img src="queen-captures.png">

We will solve this puzzle by coding it as a formula of propositional logic.  This formula will be solvable iff the eight queens puzzle has a solution.  We will use the algorithm of Davis and Putnam to compute the solution of this formula.

In [ ]:
%run Davis-Putnam.ipynb

The function $\texttt{var}(r, c)$ takes a row $r$ and a column $c$ and returns the string $\texttt{'Q(}r\texttt{,}c\texttt{)'}$.  This string is interpreted as a propositional variable specifying that there is a queen in row $r$ and column $c$.  The image below shows how theses variables correspond to the positions on a chess board.

<img src="queens-vars.png">

In [ ]:
def var(row, col):
    return 'Q(' + str(row) + ',' + str(col) + ')'

In [ ]:
var(2,5)

Given a set of propositional variables $S$, the function $\texttt{atMostOne}(S)$ returns a set containing a single clause that expresses the fact that **at most one** of the variables in $S$ is `True`.

In [ ]:
def atMostOne(S): 
    return { frozenset({('¬',p), ('¬', q)}) for p in S
                                            for q in S 
                                            if  p != q 
           }

In [ ]:
atMostOne({'a', 'b', 'c'})

Given a <tt>row</tt> and the size of the board $n$, the procedure $\texttt{atMostOneInRow}(\texttt{row}, n)$ computes a set of clauses that is `True` if and only there is at most one queen in $\texttt{row}$.

In [ ]:
def atMostOneInRow(row, n):
    return atMostOne({ var(row, col) for col in range(1,n+1) })

In [ ]:
atMostOneInRow(3, 4)

Given a column <tt>col</tt> and the size of the board $n$, the procedure $\texttt{oneInColumn}(\texttt{col}, n)$ computes a set of clauses that is true if and only if there is at least one queen in the column $\texttt{col}$.

In [ ]:
def oneInColumn(col, n):
    return { frozenset({ var(row, col) for row in range(1,n+1) }) }

In [ ]:
oneInColumn(2, 4)

Given a number $k$ and the size of the board $n$, the procedure $\texttt{atMostOneInFallingDiagonal}(k, n)$ computes a set of clauses that is `True` if and only if there is at most one queen in the falling diagonal specified by the equation
$$ \texttt{row} - \texttt{col} = k. $$

In [ ]:
def atMostOneInFallingDiagonal(k, n):
    S = { var(row, col) for row in range(1, n+1)
                        for col in range(1, n+1) 
                        if  row - col == k 
        }
    return atMostOne(S)

In [ ]:
atMostOneInFallingDiagonal(0, 4)

Given a number $k$ and the size of the board $n$, the procedure $\texttt{atMostOneInRisingDiagonal}(k, n)$ computes a set of clauses that is `True` if and only if there is at most one queen in the rising diagonal specified by the equation
$$ \texttt{row} + \texttt{col} = k. $$

In [ ]:
def atMostOneInRisingDiagonal(k, n):
    S = { var(row, col) for row in range(1, n+1)
                        for col in range(1, n+1) 
                        if  row + col == k 
        }
    return atMostOne(S)

In [ ]:
atMostOneInRisingDiagonal(3, 4)

The function $\texttt{allClauses}(n)$ takes the size of the board $n$ and computes a set of clauses that specify that
<ol>
    <li>there is at most one queen in every row,</li>
    <li>there is at most one queen in every rising diagonal,</li>
    <li>there is at most one queen in every falling diagonal, and</li>
    <li>there is at least one queen in every column.</li>
</ol>

In [ ]:
def allClauses(n):
    All = [ atMostOneInRow(row, n)           for row in range(1, n+1)        ] \
        + [ atMostOneInRisingDiagonal(k, n)  for k in range(3, (2*n-1)+1)    ] \
        + [ atMostOneInFallingDiagonal(k, n) for k in range(-(n-2), (n-2)+1) ] \
        + [ oneInColumn(col, n)              for col in range(1, n+1)        ]
    return { clause for S in All for clause in S }

In [ ]:
for C in allClauses(8):
    print(set(C))

The set of all clauses contains 512 clauses.  There are 64 variables.  

In [ ]:
len(allClauses(8))

The function $\texttt{printBoard}(I, n)$ takes a set of unit clauses $I$ that represents a propositional valuation solving the $n$ queens problem and prints the solution represented by $I$.

In [ ]:
def printBoard(I, n):
    if I == { frozenset() }:
        return
    print("-" * (8*n+1))
    for row in range(1, n+1):
        printEmptyLine(n)
        line = "|";
        for col in range(1, n+1):
            if frozenset({ var(row, col) }) in I:
                line += "   Q   |"
            else:
                line += "       |"
        print(line)
        printEmptyLine(n)
        print("-" * (8*n+1))

def printEmptyLine(n):
    line = "|"
    for col in range(1, n+1):
        line += "       |"
    print(line)

The function $\texttt{queens}(n)$ solves the n queens problem.

In [ ]:
def queens(n):
    "Solve the n queens problem."
    Clauses  = allClauses(n)
    Solution = solve(Clauses, set())
    if Solution != { frozenset() }:
        return Solution
    else:
        print(f'The problem is not solvable for {n} queens!')

In [ ]:
%%time 
Solution = queens(8)

The fact that it takes less than a second to solve the 8 queens puzzle demonstrates the efficiency of the Davis Putnam procedure.

In [ ]:
printBoard(Solution, 8)

In order to have a more convenient view of the solution, we have to install `python-chess`.  
This can be done using the following command:


In [ ]:
!pip install python-chess

In [ ]:
import chess

This function takes a solution, which is represented as a set of unit clauses and displays it as a chess board with n queens

In [ ]:
def show_solution(Solution, n):
    board = chess.Board(None)  # create empty chess board
    queen = chess.Piece(chess.QUEEN, True)
    for row in range(1, n+1):
        for col in range(1, n+1):
            field_number = (row - 1) * 8 + col - 1
            if frozenset({ var(row, col) }) in Solution:
                board.set_piece_at(field_number, queen)
    display(board)

In [ ]:
show_solution(Solution, 8)